In [4]:
pip install torch torchvision matplotlib


Note: you may need to restart the kernel to use updated packages.


In [5]:
from roboflow import Roboflow

rf = Roboflow(api_key="hY9qOmC03Dpg4JNVNeOp")
project = rf.workspace("random-gauma").project("valorant-lobqc")  # or your project name
version = project.version(2)
dataset = version.download("folder")  # ✅ Classification format


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to valorant-2 in folder:: 100%|██████████| 86/86 [00:00<00:00, 3828.26it/s]


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
data_dir = "valorant-2"  # adjust if your folder name is different

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
valid_dataset = ImageFolder(root=os.path.join(data_dir, "valid"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

# Load pre-trained model
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))  # 2 classes
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # you can increase to 10+ for better results
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader):.4f}")

# Save model
torch.save(model.state_dict(), "phase_classifier.pt")
print("✅ Model saved as phase_classifier.pt")


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Arkin Kansra/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 48.5MB/s]


Epoch 1 Loss: 0.3083
Epoch 2 Loss: 1.7019
Epoch 3 Loss: 0.4122
Epoch 4 Loss: 0.9454
Epoch 5 Loss: 0.0560
Epoch 6 Loss: 0.2729
Epoch 7 Loss: 0.0985
Epoch 8 Loss: 0.0102
Epoch 9 Loss: 0.0743
Epoch 10 Loss: 0.0102
✅ Model saved as phase_classifier.pt
